# Assignment Solution: Compare Song Lyrics



# Imports

In [ ]:
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import re

# Task 1: Load GloVe Vectos

In [ ]:
# https://nlp.stanford.edu/projects/glove/

In [ ]:
glove_zip_path = "/content/drive/MyDrive/Colab Notebooks/glove.6B.zip"

In [ ]:
with zipfile.ZipFile(glove_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/glove')

In [ ]:
glove_file_path = '/content/glove/glove.6B.300d.txt'

In [ ]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            weights = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = weights
    return embeddings_index

embeddings_index = load_glove_embeddings(glove_file_path)

# Task 2: Till I Collapse ve Walk txt dosyalarını okuyunuz ve kelimelerine ayırınız.

In [ ]:
def read_lyrics(file_path):
    with open(file_path, 'r') as file:
        lyrics = file.read().replace('\n', ' ')
    return lyrics

In [ ]:
lyrics1 = read_lyrics('/content/till_i_collapse.txt')
lyrics2 = read_lyrics('/content/walk.txt')

In [ ]:
lyrics1

"(Yo, left, yo, left) 'cause sometimes you just feel tired, feel weak (Yo, left, right, left) and when you feel weak (Yo, left, yo, left) you feel like you wanna just give up (Yo, left, right, left) but you gotta search within you (Yo, left, yo, left) try to find that inner strength and just pull that shit out of you (Yo, left, right, left) and get that motivation to not give up (Yo, left, yo, left) and not be a quitter, no matter how bad (Yo, left, right, left) you wanna just fall flat on your face and collapse  Till I collapse, I'm spillin' these raps long as you feel 'em Till the day that I drop, you'll never say that I'm not killin' 'em 'Cause when I am not, then I'ma stop pennin' 'em And I am not hip-hop and I'm just not Eminem Subliminal thoughts, when I'ma stop sendin' 'em? Women are caught in webs, spin 'em and hock venom Adrenaline shots of penicillin could not get the illin' to stop Amoxicillin's just not real enough The criminal, cop killin', hip-hop villain A minimal swap t

In [ ]:
lyrics2

"Used to run down this road With my heart on fire Now I'm sitting right here by the side Like a bird on the wire See my dreams passing by And I realize The day that I quit Is the day I expire Woah-oh-oh, I gotta keep moving on, gotta keep moving on, oh-oh Oh-oh-oh, I'm shaking the dirt from my bones All I gotta do is Walk I'm fighting and I'm bleeding I don't give up, I walk My hope, my love, my reason They guide me as I walk Like Hannibal over the mountains I gotta keep moving, I won't stop Keep on walking, watch out Here I'm coming Walk (walk), walk (walk) Walk (walk), walk (walk) Ain't no knocking me over I keep bouncing back Now I'm done with the noise in my head Yeah, I follow my track A battle for gold And a dream in my soul In times when I fail I shake off the pain and Woah-oh-oh, I gotta keep moving on, gotta keep moving on, oh-oh Oh-oh-oh, I'm shaking the dirt from my bones And all I gotta do is Walk I'm fighting and I'm bleeding I don't give up, I walk My hope, my love, my re

# Task 3: Bu 2 lyrics dosyasını veri ön işlemeden geçiriniz.

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # punctuations
    text = text.lower()  # lower case
    text = re.sub(r'\d+', '', text)  # numbers
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Stopwords
    text = re.sub(r'\s+', ' ', text).strip()  # space
    return text



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
lyrics1 = clean_text(lyrics1)
lyrics2 = clean_text(lyrics2)

In [ ]:
lyrics1

'yo left yo left cause sometimes feel tired feel weak yo left right left feel weak yo left yo left feel like wanna give yo left right left gotta search within yo left yo left try find inner strength pull shit yo left right left get motivation give yo left yo left quitter matter bad yo left right left wanna fall flat face collapse till collapse im spillin raps long feel em till day drop youll never say im killin em cause ima stop pennin em hiphop im eminem subliminal thoughts ima stop sendin em women caught webs spin em hock venom adrenaline shots penicillin could get illin stop amoxicillins real enough criminal cop killin hiphop villain minimal swap cop millions pac listeners youre comin feel youre gonna fear like showed ya spirit god lives us hear lot lyrics shock miracle product pop fizzin fa shizzle wizzle plot listen bizzles forgot slizzle give fuck till roof comes till lights go till legs give cant shut mouth till smoke clears high perhaps ima rip shit till bones collapse till roo

In [ ]:
lyrics2

'used run road heart fire im sitting right side like bird wire see dreams passing realize day quit day expire woahohoh gotta keep moving gotta keep moving ohoh ohohoh im shaking dirt bones gotta walk im fighting im bleeding dont give walk hope love reason guide walk like hannibal mountains gotta keep moving wont stop keep walking watch im coming walk walk walk walk walk walk walk walk aint knocking keep bouncing back im done noise head yeah follow track battle gold dream soul times fail shake pain woahohoh gotta keep moving gotta keep moving ohoh ohohoh im shaking dirt bones gotta walk im fighting im bleeding dont give walk hope love reason guide walk like hannibal mountains gotta keep moving wont stop keep walking watch im coming walk walk walk walk walk walk walk walk watch walk walk walk walk walk gotta keep moving wont stop keep walking watch im coming sitting waiting better days facing battle fight change ive made decision clearing haze times fail im taking pain im way walk im fig

# Task 4: 2 ayrı lyrics dosyası için embeddings_index içerisinden kelimelerin vektörlerini getiriniz.

In [ ]:
lyrics1_words = lyrics1.split()
lyrics1_vectors = [embeddings_index[word] for word in lyrics1_words if word in embeddings_index]

In [ ]:
lyrics1_words[0:10]

['yo',
 'left',
 'yo',
 'left',
 'cause',
 'sometimes',
 'feel',
 'tired',
 'feel',
 'weak']

In [ ]:
lyrics1_vectors[0:3]

[array([ 4.6120e-01, -4.0179e-01, -1.3730e-01,  5.2212e-01, -9.4870e-01,
        -2.2099e-01, -1.2509e+00, -9.2991e-01,  1.4156e-01,  2.5852e-01,
         2.7787e-01, -1.7296e-01, -2.6931e-01,  2.0884e-01,  1.5174e-01,
        -2.7084e-01, -4.5824e-01,  8.1970e-01,  5.2072e-01,  5.3009e-01,
        -2.1830e-01,  1.9101e-01, -5.9654e-02, -5.4589e-01, -1.5450e-01,
         2.7008e-01,  3.9233e-01, -5.4322e-01,  2.4878e-01,  1.9002e-01,
        -2.8343e-01, -7.8935e-02, -1.9338e-01, -3.3979e-01,  2.4354e-01,
         1.6205e-01,  1.7326e-01,  1.4640e-01, -1.7354e-01,  3.5677e-01,
        -1.9414e-01, -6.7813e-01, -6.8590e-02,  1.7963e-01, -5.5778e-01,
         5.3919e-01,  3.6288e-01, -6.1546e-01, -2.3783e-01,  5.7239e-02,
         5.4897e-01, -4.5660e-01, -3.4407e-01, -3.2051e-02,  3.0226e-02,
        -3.3490e-02, -4.2817e-02, -3.5129e-01,  5.2198e-02, -1.0461e+00,
         4.0872e-01,  4.9958e-01,  2.7968e-01,  3.2315e-01,  4.5589e-01,
         2.7989e-01, -4.9916e-01,  3.1172e-01, -3.9

In [ ]:
lyrics2_words = lyrics2.split()
lyrics2_vectors = [embeddings_index[word] for word in lyrics2_words if word in embeddings_index]

In [ ]:
lyrics2_vectors[0:3]

[array([-0.36655  ,  0.4255   , -0.039372 , -0.50503  , -0.02789  ,
        -0.015668 , -0.10517  ,  0.35168  , -0.16999  , -2.0578   ,
         0.29368  , -0.15804  ,  0.084983 ,  0.017398 ,  0.281    ,
         0.082806 , -0.23189  ,  0.16941  , -0.13253  , -0.59823  ,
         0.041188 , -0.39048  ,  0.0031465,  0.15583  , -0.054264 ,
        -0.2558   , -0.051299 ,  0.14238  , -0.11393  ,  0.39564  ,
         0.10386  ,  0.30817  , -0.01106  ,  0.6111   , -0.12979  ,
        -0.2306   , -0.14391  , -0.1878   ,  0.062193 , -0.18817  ,
        -0.35477  ,  0.31697  , -0.33145  , -0.32406  ,  0.29054  ,
         0.17762  , -0.15771  ,  0.1737   , -0.18423  ,  0.30411  ,
         0.079133 ,  0.056365 ,  0.44119  , -0.11082  , -0.061754 ,
        -0.19426  , -0.18083  ,  0.073888 , -0.026451 ,  0.1245   ,
        -0.05933  ,  0.16679  ,  0.55378  ,  0.20749  ,  0.26545  ,
        -0.25298  , -0.46913  ,  0.19056  ,  0.45376  ,  0.34137  ,
         0.079838 ,  0.63634  , -0.036132 ,  0.3

# Task 5: Vektörlerin ortalamalarını alınız

In [ ]:
lyrics1_average_vector = np.mean(lyrics1_vectors, axis=0).reshape(1, -1)

In [ ]:
lyrics2_average_vector = np.mean(lyrics2_vectors, axis=0).reshape(1, -1)

In [ ]:
lyrics2_average_vector

array([[ 3.98621336e-03, -8.44659284e-02, -4.55612279e-02,
         7.21670017e-02, -1.69714898e-01, -4.57000965e-03,
         5.94793307e-03, -7.48961642e-02,  1.03674516e-01,
        -9.27506030e-01,  1.27358824e-01, -2.41271481e-01,
        -3.07302535e-01, -1.10279091e-01,  4.45311889e-03,
         3.94368134e-02, -1.60431549e-01,  3.48119438e-03,
         2.27310553e-01,  1.79678619e-01,  1.25121951e-01,
         9.31046680e-02,  1.82495251e-01,  4.52192836e-02,
        -4.13993560e-02, -1.00848161e-01,  1.63074687e-01,
        -8.42178017e-02,  1.82313491e-02,  8.79410729e-02,
         1.90217137e-01,  1.05172358e-01, -4.80545908e-02,
        -1.68881297e-01, -7.61925936e-01,  1.30317554e-01,
        -1.97554111e-01,  6.51252968e-03, -1.45421684e-01,
        -8.07868168e-02, -1.21489586e-02, -9.11126137e-02,
        -1.34781420e-01,  1.19905315e-01,  2.28775106e-02,
         1.08596608e-01,  3.79018784e-01, -2.17695441e-02,
         2.81792767e-02, -3.51923406e-02, -4.36107302e-0

# Task 6: Cosine Similarity hesaplayınız.

In [ ]:
similarity = cosine_similarity(lyrics1_average_vector, lyrics2_average_vector)
print(f"Similarity Between 2 Songs: {similarity[0][0]}")

Similarity Between 2 Songs: 0.8631009459495544
